# Autocfg Basics

This notebook will demonstrate the basic functionalities of autocfg

In [1]:
from autocfg import dataclass, field  # drop-in replacement of dataclass decorator out of dataclasses

### dataclass decorator

The usage of dataclass decorator shouldn't be anything different than the native `dataclasses` introduced in python 3.7. In python 3.6 we use the backported `dataclasses` so it minimum requirement of `autocfg` package is python 3.6

Let's create some random configurations you will use in an experiment.

In [2]:
# first is the common training config
@dataclass
class TrainConfig:
  batch_size : int = 32
  learning_rate : float = 1e-3
  weight_decay : float = 1e-5

followed by a nested config

In [3]:
# supports nested config
@dataclass
class MyExp:
  train : TrainConfig = field(default_factory=TrainConfig)
  num_class : int = 1000
  depth : int = 50

Note that it's very important to keep nested config's default a `field` with `default_factory` rather than a default value `TrainConfig()`, since the default values is mutable and can be overwriten inaccidentally.

### Initialize, and direct access

In [4]:
# we can initialize the plain configs as-is
train = TrainConfig()
train1 = TrainConfig(batch_size=128)
print('train:', train)
print('train1:', train1)

train: TrainConfig(batch_size=32, learning_rate=0.001, weight_decay=1e-05)
train1: TrainConfig(batch_size=128, learning_rate=0.001, weight_decay=1e-05)


In [5]:
# the exp config, a nested class
exp = MyExp(depth=18, train=TrainConfig())
print('exp with default train:', exp)

exp with default train: MyExp(train=TrainConfig(batch_size=32, learning_rate=0.001, weight_decay=1e-05), num_class=1000, depth=18)


In [6]:
# config can be viewed as normal dict
print('dict:', exp.asdict())

dict: {'train': {'batch_size': 32, 'learning_rate': 0.001, 'weight_decay': 1e-05}, 'num_class': 1000, 'depth': 18}


In [7]:
# To modify the values, attributes can be directly accessed
exp.num_class = 10
exp.train.learning_rate = 1.5
print('updated exp:', exp)

updated exp: MyExp(train=TrainConfig(batch_size=32, learning_rate=1.5, weight_decay=1e-05), num_class=10, depth=18)


### Serialization

`autocfg` prefers `yaml` as the human-readable format for serialization, which can be viewed and modified pretty effortlessly.

In [8]:
# save to 'exp.yaml'
exp.save('exp.yaml')
!cat exp.yaml

# MyExp
depth: 18
num_class: 10
train:
  batch_size: 32
  learning_rate: 1.5
  weight_decay: 1.0e-05


In [9]:
# directly load from file is also straight-forward
exp1 = MyExp.load('exp.yaml')
assert exp == exp1

In [10]:
# a python file-like object can be handy in case in-memory operation is preferred
import io
f = io.StringIO('depth: 1000')
exp2 = MyExp.load(f)
print(exp2)
assert exp2.depth == 1000

MyExp(train=TrainConfig(batch_size=32, learning_rate=0.001, weight_decay=1e-05), num_class=1000, depth=1000)


### Update config
Though configs can be updated by direct access and assignment, we also need a faster `update` method similar to nested dict

In [11]:
exp2.update(exp1)
print(exp2)
assert exp2 == exp1

MyExp(train=TrainConfig(batch_size=32, learning_rate=1.5, weight_decay=1e-05), num_class=10, depth=18)


In [12]:
# update support files, file-like objects where configs has been dumped
exp2 = MyExp(num_class=200)
exp2.update('exp.yaml')
print(exp2)
assert exp2 == exp

MyExp(train=TrainConfig(batch_size=32, learning_rate=1.5, weight_decay=1e-05), num_class=10, depth=18)


In [13]:
# update with a dict
exp2.update({'num_class': 10, 'train': {'learning_rate': 1.0}})
print(exp2)
assert exp2.num_class == 10 and exp2.train.learning_rate == 1.0

MyExp(train=TrainConfig(batch_size=32, learning_rate=1.0, weight_decay=1e-05), num_class=10, depth=18)


### Argparse integration

It's always time consuming if you need to manually add a argparse parser to handle command line inputs, `autocfg` handles the issue out of the box, saving tons of efforts for you.

Since jupyter notebook doesnot play with `sys.argv` well, we will use list inputs to simulate command line inputs.

In [14]:
# auto generated helper
try:
    new_exp = MyExp.parse_args(['-h'])
except SystemExit as e:
    pass

usage: MyExp's auto argument parser [-h] [--train.batch-size TRAIN.BATCH_SIZE]
                                    [--train.learning-rate TRAIN.LEARNING_RATE]
                                    [--train.weight-decay TRAIN.WEIGHT_DECAY]
                                    [--num-class NUM_CLASS] [--depth DEPTH]

optional arguments:
  -h, --help            show this help message and exit
  --train.batch-size TRAIN.BATCH_SIZE
                        batch_size (default: 32)
  --train.learning-rate TRAIN.LEARNING_RATE
                        learning_rate (default: 0.001)
  --train.weight-decay TRAIN.WEIGHT_DECAY
                        weight_decay (default: 1e-05)
  --num-class NUM_CLASS
                        num_class (default: 1000)
  --depth DEPTH         depth (default: 50)


The default values are also available in the console output

In [15]:
# normal overriding
new_exp = MyExp.parse_args(['--depth', '100'])
print(new_exp)

MyExp(train=TrainConfig(batch_size=32, learning_rate=0.001, weight_decay=1e-05), num_class=1000, depth=100)


In [16]:
# nested overriding
new_exp = MyExp.parse_args(['--train.weight-decay', '100.0'])
print(new_exp)

MyExp(train=TrainConfig(batch_size=32, learning_rate=0.001, weight_decay=100.0), num_class=1000, depth=50)


### Diff configurations

Knowing what's been modified is an important feature of configuration systems, `autocfg` provide a `diff` function to evaluate the changes

In [17]:
from pprint import pprint
pprint(new_exp.diff(exp2))

['root.depth           50 != 18',
 'root.train.weight_decay 100.0 != 1e-05',
 'root.train.learning_rate 0.001 != 1.0',
 'root.num_class       1000 != 10']
